# Let's compile a model trained on mnist with MATCH
The model can be found in the model directory and has been pre-trained, quantized and integerized with PLiNIO https://github.com/eml-eda/plinio to int32 to target ARCANE.
This model is quantized to int8, and is comprised by a set of Dense(MatMul)+BatchNorm(multiply+add) operations, thanks to PLiNIO Dense Weigths and BatchNorm constants are trained to decrease the error, also additionally there is a Div operation, that serves as a rescale factor. 
In the PLiNIO repository there are examples to reproduce the model directory, where we have our input, the expected label and finally also the model itself.
In this case we'll compile the model for the default target where only the host-cpu will be used.

## Setup
First let's import all the required libraries and setup correctly the environment

In [ ]:
# setup environment variables of MATCH and TVM
import os
CURR_PATH = "./"
MATCH_PATH = "./../../../"
TVM_HOME = f"{MATCH_PATH}/match-tvm"
os.environ["TVM_HOME"] = TVM_HOME
os.environ["PYTHONPATH"] = f"{TVM_HOME}/python:{MATCH_PATH}/zigzag:{os.environ['PYTHONPATH']}"
# MATCH imports
import match
from match.target.target import DefaultMatchTarget
from match.utils.utils import get_default_inputs
from match.model.model import MatchModel
from tvm import relay
import onnx

## Match Model
Now let's define the model in MATCH, these are the parameters used for MatchModel:
- filename: ONNX/Relay IR file containing the model definition
- model\_name: name that will be used to identify the model in case of multi-model compilation or more complex situations
- default\_inputs: inputs which will be used to compile the model, they will be stored in src(include)/model\_name/default_input.c(h), by default the model will run with this inputs if not specified otherwise
- handle\_out\_fn: function used to handle the output after a single inference(with the default inputs), in this case we'll use a MATCH-provided one that handle int32 classification, but a user-specified one can be put


In [ ]:
input_file = CURR_PATH+"/model/input.txt"
onnx_file_path = CURR_PATH+"/model/mnist.onnx"
onnx_model = onnx.load(onnx_file_path)
mod, _ = relay.frontend.from_onnx(onnx_model)
mnist_model = MatchModel(
    filename=onnx_file_path,
    model_name="mnist",
    default_inputs=get_default_inputs(mod=mod, input_files=[input_file]),
    handle_out_fn="handle_int_classifier"
)

## Compile the MatchModel
Finally with the MatchModel available we can compile it with MATCH for the provided target!

In [ ]:
match.match(
    model=mnist_model,
    target=DefaultMatchTarget(),
    output_path=CURR_PATH+"/output",
)

## Compilation result
Now we can check the graph of the network in Relay IR(TVM lower level IR to represent a network).
After we can compile it to binary and run it to check if we obtain the expected result

In [ ]:
!cat output/models/mnist/relay/partitioned_graph.relay

In [ ]:
!cd output; make all; ./match_app

The expected label should be:

In [ ]:
!cat model/label.txt